# Challenge 4 study - Quantized Model with Conv Input Layer

## Import libraries and data, init virtual device

In [13]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, ReLU, Conv2D, Flatten

from akida import Device, AKD1000
import cnn2snn

device = AKD1000()

## Data Preparation

In [14]:
coords_ch4 = np.load('coords_ch4.npy')
color = np.load('result_ch4.npy')

# Pack in a dataframe and scale for 8-bits quantization (first layer will be quantized on 8-bits)
data = pd.DataFrame(coords_ch4, columns = ['x', 'y'])
data = round(255*data).astype(np.uint8)


# Shuffle and split train/test
data['color'] = color
data_train = data.sample(frac=0.7, axis=0)
data_test = data.drop(data_train.index)

# Split x y
x_train = data_train.drop('color', axis=1)
y_train = data_train['color']
x_test = data_test.drop('color', axis=1)
y_test = data_test['color']


# Attempt to encode coordinates as an image: put the x value as grey scale on the left, then y on the right
left = np.empty((6, 3))
right = np.empty((6, 3))

x_train_coded = np.empty((7000, 6, 6))
for i, p in enumerate(x_train.to_numpy()):
    left.fill(p[0])
    right.fill(p[1])
    x_train_coded[i] = np.concatenate((left, right), axis=1)
x_train_coded = x_train_coded.reshape(-1, 6, 6, 1)

x_test_coded = np.empty((3000, 6, 6))
for i, p in enumerate(x_test.to_numpy()):
    left.fill(p[0])
    right.fill(p[1])
    x_test_coded[i] = np.concatenate((left, right), axis=1)
x_test_coded = x_test_coded.reshape(-1, 6, 6, 1)

## Model creation with Keras

In [15]:
base = Sequential(name="Base_ch4")
base.add(Conv2D(1, (3, 3), strides=(3, 3), input_shape=(6, 6, 1), name='C2D'))
base.add(ReLU(name='relu0'))
base.add(Flatten())
base.add(Dense(1000, name='FC1'))
base.add(ReLU(name='relu1'))
base.add(Dense(500, name='FC2'))
base.add(ReLU(name='relu2'))
base.add(Dense(250, name='FC3'))
base.add(ReLU(name='relu3'))
base.add(Dense(100, name='FC4'))
base.add(ReLU(name='relu4'))
base.add(Dense(50, name='FC5'))
base.add(ReLU(name='relu5'))
base.add(Dense(1, name='FC6'))
base.add(Activation('sigmoid', name='sigmoid'))

#base.summary()

## Model quantization (4-bits, input: 8-bit)

In [16]:
qmodel = cnn2snn.quantize(base, weight_quantization=4, activ_quantization=4, input_weight_quantization=8)
qmodel.summary()

print('\nAkida Compatibility:', cnn2snn.check_model_compatibility(qmodel, input_is_image=True))

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C2D (QuantizedConv2D)       (None, 2, 2, 1)           10        
                                                                 
 relu0 (QuantizedReLU)       (None, 2, 2, 1)           0         
                                                                 
 flatten_2 (Flatten)         (None, 4)                 0         
                                                                 
 FC1 (QuantizedDense)        (None, 1000)              5000      
                                                                 
 relu1 (QuantizedReLU)       (None, 1000)              0         
                                                                 
 FC2 (QuantizedDense)        (None, 500)               500500    
                                                                 
 relu2 (QuantizedReLU)       (None, 500)             

## Preview akida model

In [17]:
akmodel = cnn2snn.convert(qmodel, input_is_image=True)
akmodel.map(device)
akmodel.summary()

                   Model Summary                   
___________________________________________________
Input shape  Output shape  Sequences  Layers  NPs
[6, 6, 1]    [1, 1, 1]     1          7       6  
___________________________________________________

________________________________________________________
Layer (type)      Output shape  Kernel shape       NPs

====== HW/C2D-FC6 (Hardware) - size: 560560 bytes ======

C2D (InputConv.)  [2, 2, 1]     (3, 3, 1, 1)       N/A
________________________________________________________
FC1 (Fully.)      [1, 1, 1000]  (1, 1, 4, 1000)    1  
________________________________________________________
FC2 (Fully.)      [1, 1, 500]   (1, 1, 1000, 500)  1  
________________________________________________________
FC3 (Fully.)      [1, 1, 250]   (1, 1, 500, 250)   1  
________________________________________________________
FC4 (Fully.)      [1, 1, 100]   (1, 1, 250, 100)   1  
________________________________________________________
FC5 (Fully.)

## Train quantized model

In [18]:
qmodel.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
            )
qmodel.fit(x_train_coded, y_train, batch_size=64, epochs=600)
qmodel.evaluate(x_test_coded, y_test)

Epoch 1/600
110/110 [==============================] - 6s 28ms/step - loss: 0.6932 - accuracy: 0.4977
Epoch 2/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4997
Epoch 3/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4963
Epoch 4/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4923
Epoch 5/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4991
Epoch 6/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4920
Epoch 7/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4966
Epoch 8/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4940
Epoch 9/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4877
Epoch 10/600
110/110 [==============================] - 3s 28ms/step - loss: 0.693

110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4917
Epoch 81/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 82/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4891
Epoch 83/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4949
Epoch 84/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4963
Epoch 85/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4940
Epoch 86/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4951
Epoch 87/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 88/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6932 - accuracy: 0.4954
Epoch 89/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - 

110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 160/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 161/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4951
Epoch 162/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6932 - accuracy: 0.4891
Epoch 163/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4954
Epoch 164/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4923
Epoch 165/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4920
Epoch 166/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4863
Epoch 167/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4957
Epoch 168/600
110/110 [==============================] - 3s 29ms/step - loss: 

110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4971
Epoch 238/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4954
Epoch 239/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4940
Epoch 240/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4946
Epoch 241/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4989
Epoch 242/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 243/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4966
Epoch 244/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6932 - accuracy: 0.4923
Epoch 245/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4943
Epoch 246/600
110/110 [==============================] - 3s 29ms/step - loss: 

110/110 [==============================] - 4s 32ms/step - loss: 0.6932 - accuracy: 0.4974
Epoch 316/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4914
Epoch 317/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4909
Epoch 318/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 319/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4963
Epoch 320/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4969
Epoch 321/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 322/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 323/600
110/110 [==============================] - 3s 31ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 324/600
110/110 [==============================] - 3s 28ms/step - loss: 

110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4983
Epoch 394/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4940
Epoch 395/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4989
Epoch 396/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4914
Epoch 397/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4923
Epoch 398/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4963
Epoch 399/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 400/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4969
Epoch 401/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4949
Epoch 402/600
110/110 [==============================] - 3s 29ms/step - loss: 

110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4980
Epoch 472/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 473/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 474/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 475/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4957
Epoch 476/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4943
Epoch 477/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4914
Epoch 478/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 479/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4963
Epoch 480/600
110/110 [==============================] - 3s 30ms/step - loss: 

110/110 [==============================] - 3s 30ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 550/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4974
Epoch 551/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 552/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.5003
Epoch 553/600
110/110 [==============================] - 3s 28ms/step - loss: 0.6932 - accuracy: 0.4940
Epoch 554/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6932 - accuracy: 0.4960
Epoch 555/600
110/110 [==============================] - 3s 29ms/step - loss: 0.6932 - accuracy: 0.4914
Epoch 556/600
110/110 [==============================] - 4s 38ms/step - loss: 0.6932 - accuracy: 0.4960
Epoch 557/600
110/110 [==============================] - 3s 30ms/step - loss: 0.6932 - accuracy: 0.4900
Epoch 558/600
110/110 [==============================] - 3s 29ms/step - loss: 

[0.693136990070343, 0.5053333044052124]